# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#Data extraction
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hilo,20,US,1612983720,68,19.7297,-155.0900,71.01,5.75
1,Puerto Ayora,75,EC,1612983808,83,-0.7393,-90.3518,80.60,3.44
2,Dikson,94,RU,1612983808,89,73.5069,80.5464,-20.83,7.96
3,Atuona,2,PF,1612983808,75,-9.8000,-139.0333,79.18,17.27
4,Kushiro,1,JP,1612983809,93,42.9750,144.3747,15.80,4.43


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [9]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
46,Port Blair,0,IN,1612983813,69,11.6667,92.7500,76.59,9.53
89,Salalah,0,OM,1612983817,25,17.0151,54.0924,78.80,6.91
217,Marsh Harbour,0,BS,1612983833,73,26.5412,-77.0636,75.56,9.26
219,Wad Medani,0,SD,1612983788,18,14.4012,33.5199,77.40,8.88
279,Hurghada,0,EG,1612983839,46,27.2574,33.8129,71.60,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
46,Port Blair,IN,11.6667,92.7500,
89,Salalah,OM,17.0151,54.0924,
217,Marsh Harbour,BS,26.5412,-77.0636,
219,Wad Medani,SD,14.4012,33.5199,
279,Hurghada,EG,27.2574,33.8129,


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [16]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 46: Port Blair.
Closest hotel in Port Blair is Welcomhotel Bay Island Port Blair.
------------
Retrieving Results for Index 89: Salalah.
Closest hotel in Salalah is HYATT SALALAH HOTEL.
------------
Retrieving Results for Index 217: Marsh Harbour.
Closest hotel in Marsh Harbour is Abaco Beach Resort.
------------
Retrieving Results for Index 219: Wad Medani.
Closest hotel in Wad Medani is Imperial Wad Medani Hotel.
------------
Retrieving Results for Index 279: Hurghada.
Closest hotel in Hurghada is Marina Star Hotel.
------------
Retrieving Results for Index 294: Hambantota.
Closest hotel in Hambantota is The Peacock Beach Hotel Hambantota.
------------
Retrieving Results for Index 302: Bilma.
Missing field/result... skipping.
------------
Retrieving Results for Index 400: Tessalit.
Missing field/result... skipping.
------------
Retrieving Results for Index 484: Luganville.
Closest hotel in Luganville is The Espiritu Hotel.
------------
Retrieving Results 

In [17]:
# Display the hotel dataframe
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
46,Port Blair,IN,11.6667,92.7500,Welcomhotel Bay Island Port Blair
89,Salalah,OM,17.0151,54.0924,HYATT SALALAH HOTEL
217,Marsh Harbour,BS,26.5412,-77.0636,Abaco Beach Resort
219,Wad Medani,SD,14.4012,33.5199,Imperial Wad Medani Hotel
279,Hurghada,EG,27.2574,33.8129,Marina Star Hotel


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

In [20]:
# Add the layer to the map
fig.add_layer(markers)

In [21]:
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))